In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
! pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [ ]:
!adk create Customer_Service_Agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [4]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent , LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner , Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool, FunctionTool, google_search , ToolContext
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.genai import types
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.apps.app import App, ResumabilityConfig
import uuid

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [5]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [7]:
billing_database = {
    "1001": {"name": "Sarah Johnson", "plan": "Premium", "amount_due": 78.50, "due_date": "2025-01-14"},
    "1002": {"name": "Mark Davis", "plan": "Unlimited", "amount_due": 55.00, "due_date": "2025-01-11"},
    "1003": {"name": "Emily Rodriguez", "plan": "Basic", "amount_due": 25.99, "due_date": "2025-01-07"},
}

def lookup_billing(customer_id: str):
    """Return billing info for a customer."""
    if customer_id not in billing_database:
        return {"error": "Customer ID not found"}
    return billing_database[customer_id]

def update_bill_amount(customer_id: str, new_amount: float):
    """Update the billing amount for a customer."""
    if customer_id not in billing_database:
        return {"error": "Customer ID not found"}
    billing_database[customer_id]["amount_due"] = new_amount
    return {
        "status": "updated",
        "customer_id": customer_id,"new_amount": new_amount,
        "new_amount": new_amount
    }

def billing_summary_for_agent(customer_id: str):
    if customer_id not in billing_database:
        return "Customer not found."

    d = billing_database[customer_id]
    return (
        f"Customer: {d['name']}\n"
        f"Plan: {d['plan']}\n"
        f"Amount Due: ${d['amount_due']}\n"
        f"Due Date: {d['due_date']}"
    )
print("✅ Billing Tools created.")

✅ Billing Tools created.


In [8]:
lookup_billing_tool = FunctionTool(lookup_billing)
update_bill_tool = FunctionTool(update_bill_amount)
billing_summary_tool = FunctionTool(billing_summary_for_agent)

In [9]:
front_desk_agent = Agent(
    name="customer_service_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="An agent that can take customer inquiries.",
    instruction="""You are a the main agent to respond to customer inquires . You will be very friendly ,
    using a humble tone ,  communicating with other agents to get the required information and solve 
    customers problem. Use provided database. If you do not have the account number yet, 
    ask the user to provide it""",
    tools=[lookup_billing_tool,update_bill_tool,billing_summary_tool],
    output_key="customer_inquires"
)

print("✅ Front Desk Agent created.")

✅ Front Desk Agent created.


In [10]:
billing_agent = Agent(
    name="BillingAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided customer inquiries: {customer_inquires}
look for the customer's data , current bill and charged amount.""",
    output_key="billing_summary",
    tools=[lookup_billing_tool,update_bill_tool,billing_summary_tool]
)

print("✅ billing_agent created.")

✅ billing_agent created.


In [11]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the inquiry is 'SOLVED', indicating that no more actions need 
    to be taken regarding a billing issue."""
    return {"status": "solved", "message": "Inquiry solved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


In [12]:
# This agent refines the inquiry based on customer response OR calls the exit_loop function.
billing_refiner_agent = Agent(
    name="BillingRefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a customer service refiner. You have a customer billing inquiry and 
    a billing summary.
    
    Customer Inquiry: {customer_inquires}
    Billing: {billing_summary}
    
    Your task is to analyze the customer request.
    - IF the inquiry is EXACTLY "SOLVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, reheck with the customer for more feedback.""",
    output_key="customer_feedback",  # It overwrites the initial inquiry with the customer feedback .
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ billing_refiner_agent created.")

✅ billing_refiner_agent created.


In [13]:
# The LoopAgent contains the agents that will run repeatedly: 
billing_inquiry_loop = LoopAgent(
    name="BillingInquiryLoop",
    sub_agents=[billing_agent, billing_refiner_agent],
    max_iterations=3,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow:
billing_inquiries = SequentialAgent(
    name="BillingInquiries",
    sub_agents=[front_desk_agent, billing_inquiry_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [14]:
runner = InMemoryRunner(agent=front_desk_agent)

print("✅ Runner created.")

✅ Runner created.


In [15]:
response = await runner.run_debug("My bill is high , please help")



 ### Created new session: debug_session_id

User > My bill is high , please help
customer_service_agent > Hello there! I'd be happy to help you with your billing concerns. Could you please provide me with your customer ID? This will allow me to access your account and investigate the matter further.


In [16]:
response = await runner.run_debug("My acount is '1005'")



 ### Continue session: debug_session_id

User > My acount is '1005'


customer_service_agent > Thank you for providing your customer ID. Please give me a moment to look up your billing information.
customer_service_agent > I apologize, but it seems I couldn't find a customer with the ID '1005'. Could you please verify the customer ID and provide it again? I want to make sure I'm looking at the correct account to help you with your billing concerns.


In [17]:
response = await runner.run_debug("1003 and What is my current PLAN")


 ### Continue session: debug_session_id

User > 1003 and What is my current PLAN


customer_service_agent > Thank you for the clarification. Please allow me a moment to retrieve your billing summary and plan details.
customer_service_agent > Your current plan is the Basic plan, and your current amount due is $25.99. Your due date is January 7, 2025.
